In [ ]:
# === IMPORTS ===
import os
import sys
import pandas as pd
import numpy as np

parent_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from model_utils import get_device, create_pipeline, TOKENIZER_NAMES
from utils import (
    calcular_resultados, apply_protocol, load_enem_dataset,
    PROTOCOL_LABELS, PROTOCOL_ORDER, generate_latex_table,
    plot_real_vs_predicted, plot_confusion_matrix
)
from config import (
    setup_colab_paths, get_save_dir, MODEL_TEMPLATES,
    TEST_YEARS, COMPETENCIES
)
from tqdm.notebook import tqdm


In [ ]:

# === SETUP PATHS ===
try:
    DRIVE_BASE_PATH = setup_colab_paths(mount_drive=True)
except:
    DRIVE_BASE_PATH = os.path.join(os.getcwd(), "results")
    os.makedirs(DRIVE_BASE_PATH, exist_ok=True)
    print(f"✓ Using local path: {DRIVE_BASE_PATH}")

SAVE_DIR = get_save_dir(DRIVE_BASE_PATH, "inferencia_fine_tuning_originals")
print(f"✓ Diretório de predições: {SAVE_DIR}")


In [ ]:
# === CARREGAR DATASET ===
from datasets import load_dataset
_, df_test = load_enem_dataset(anos_teste=TEST_YEARS)
dataset = load_dataset("laisnuto/self-collected-ENEM-dataset", split="train")
dataset = dataset.filter(lambda x: x["ano"] in TEST_YEARS)
print(f"✓ Dataset de teste carregado: {len(df_test)} redações")
df_test.head()


In [ ]:
# === CONFIGURAÇÃO DOS MODELOS ===
device = get_device()
print(f"Using device: {device}")

model_types = MODEL_TEMPLATES["originals_finetuned"]
competencias = COMPETENCIES

pipelines = {}
for model_key, model_path_template in model_types.items():
    print(f"\n📦 Carregando modelos: {model_key}")
    pipelines[model_key] = {}
    
    tokenizer_name = TOKENIZER_NAMES.get(model_key)
    
    for c in competencias:
        comp_key = f"C{c}"
        try:
            model_name = model_path_template.format(c)
            print(f"  {comp_key} | Modelo: {model_name}")
            
            pipe = create_pipeline(
                model_name,
                tokenizer_name=tokenizer_name,
                device=device,
                max_len=512
            )
            
            pipelines[model_key][comp_key] = pipe
        except Exception as e:
            print(f"  ❌ Erro ao carregar modelo {model_name}: {e}")
            pipelines[model_key][comp_key] = None

print("\n✅ Carregamento concluído!")


In [ ]:
# Previsões com modelo mbert
model_key = "mbert"

# Verificar se já existe CSV com previsões
csv_path = os.path.join(SAVE_DIR, "predicoes_mbert_finetuning_originals.csv")

if os.path.exists(csv_path):
    print(f"✅ Carregando previsões existentes de {csv_path}")
    df_base = pd.read_csv(csv_path)
    print(f"   Carregadas {len(df_base)} previsões")
else:
    print(f"🔎 Rodando previsões com {model_key}...")
    textos = dataset["texto"]
    resultados = {f"C{c}": [] for c in competencias}

    for texto in tqdm(textos, desc=f"Processando com {model_key}"):
        for c in competencias:
            comp_key = f"C{c}"
            pred = pipelines[model_key][comp_key](texto)[0]
            label_str = pred["label"]
            classe = int(label_str.replace("LABEL_", ""))
            nota = classe * 40
            resultados[comp_key].append(nota)

    # Criar nova versão do dataset com predições
    df_base = dataset.to_pandas().copy()
    for c in competencias:
        comp_key = f"C{c}"
        df_base[f"pred_{comp_key}"] = resultados[comp_key]

    df_base["nota_final_predita"] = df_base[[f"pred_C{c}" for c in competencias]].sum(axis=1)

    # Salvar no Google Drive
    df_base.to_csv(csv_path, index=False)
    print(f"✅ Previsões salvas em: {csv_path}")



In [ ]:
# Previsões com modelo bertugues
model_key = "bertugues"

# Verificar se já existe CSV com previsões
csv_path = os.path.join(SAVE_DIR, "predicoes_bertugues_finetuning_originals.csv")

if os.path.exists(csv_path):
    print(f"✅ Carregando previsões existentes de {csv_path}")
    df_bertugues = pd.read_csv(csv_path)
    print(f"   Carregadas {len(df_bertugues)} previsões")
else:
    print(f"🔎 Rodando previsões com {model_key}...")
    textos = dataset["texto"]
    resultados = {f"C{c}": [] for c in competencias}

    for texto in tqdm(textos, desc=f"Processando com {model_key}"):
        for c in competencias:
            comp_key = f"C{c}"
            pred = pipelines[model_key][comp_key](texto)[0]
            label_str = pred["label"]
            classe = int(label_str.replace("LABEL_", ""))
            nota = classe * 40
            resultados[comp_key].append(nota)

    # Criar nova versão do dataset com predições
    df_bertugues = dataset.to_pandas().copy()
    for c in competencias:
        comp_key = f"C{c}"
        df_bertugues[f"pred_{comp_key}"] = resultados[comp_key]

    df_bertugues["nota_final_predita"] = df_bertugues[[f"pred_C{c}" for c in competencias]].sum(axis=1)

    # Salvar no Google Drive
    df_bertugues.to_csv(csv_path, index=False)
    print(f"✅ Previsões salvas em: {csv_path}")


In [ ]:
# Previsões com modelo bertimbau
model_key = "bertimbau"

# Verificar se já existe CSV com previsões
csv_path = os.path.join(SAVE_DIR, "predicoes_bertimbau_finetuning_originals.csv")

if os.path.exists(csv_path):
    print(f"✅ Carregando previsões existentes de {csv_path}")
    df_bertimbau = pd.read_csv(csv_path)
    print(f"   Carregadas {len(df_bertimbau)} previsões")
else:
    print(f"🔎 Rodando previsões com {model_key}...")
    textos = dataset["texto"]
    resultados = {f"C{c}": [] for c in competencias}

    for texto in tqdm(textos, desc=f"Processando com {model_key}"):
        for c in competencias:
            comp_key = f"C{c}"
            pred = pipelines[model_key][comp_key](texto)[0]
            label_str = pred["label"]
            classe = int(label_str.replace("LABEL_", ""))
            nota = classe * 40
            resultados[comp_key].append(nota)

    # Criar nova versão do dataset com predições
    df_bertimbau = dataset.to_pandas().copy()
    for c in competencias:
        comp_key = f"C{c}"
        df_bertimbau[f"pred_{comp_key}"] = resultados[comp_key]

    df_bertimbau["nota_final_predita"] = df_bertimbau[[f"pred_C{c}" for c in competencias]].sum(axis=1)

    # Salvar no Google Drive
    df_bertimbau.to_csv(csv_path, index=False)
    print(f"✅ Previsões salvas em: {csv_path}")


In [ ]:
# === AVALIAÇÃO COM PROTOCOLO dup_bounds ===
import ast

avaliacoes_por_modelo = {}
model_var_map = {
    "mbert": "df_base",
    "bertugues": "df_bertugues",
    "bertimbau": "df_bertimbau"
}

for model_key in ["mbert", "bertugues", "bertimbau"]:
    print(f"\n📊 Avaliando modelo: {model_key}")

    df_resultados = None
    var_name = model_var_map.get(model_key)
    
    if var_name and var_name in globals():
        try:
            df_resultados = globals()[var_name].copy()
        except NameError:
            pass

    if df_resultados is None:
        try:
            csv_path = os.path.join(SAVE_DIR, f"predicoes_{model_key}_finetuning_originals.csv")
            df_resultados = pd.read_csv(csv_path)
            print(f"  ℹ️ Usando dados do CSV")
        except (FileNotFoundError, pd.errors.EmptyDataError) as e:
            print(f"⚠️ Não foi possível encontrar dados para {model_key}. Pulando.")
            continue

    if "notas" in df_resultados.columns:
        df_resultados["notas"] = df_resultados["notas"].apply(
            lambda x: ast.literal_eval(x) if isinstance(x, str) else x
        )

    avaliacoes = {}

    for c in competencias:
        comp_key = f"C{c}"
        if "notas" in df_resultados.columns:
            y_real = df_resultados["notas"].apply(lambda x: x[c-1] if isinstance(x, (list, tuple)) else np.nan)
        else:
            y_real = df_resultados[comp_key] if comp_key in df_resultados.columns else pd.Series(dtype=float)
        
        y_pred = df_resultados[f"pred_{comp_key}"]

        y_r, y_p = apply_protocol(y_real.tolist(), y_pred.tolist(), "dup_bounds")

        if not y_r:
            print(f"⚠️ Nenhum dado válido para {comp_key}")
            continue

        resultados = calcular_resultados(y_r, y_p, qwk_step=40)
        avaliacoes[comp_key] = resultados

    avaliacoes_por_modelo[model_key] = avaliacoes

    for comp_key, resultado in avaliacoes.items():
        print(f"\n🔎 Avaliação - {comp_key}")
        print(f"  Acurácia       (ACC): {resultado['ACC']*100:.2f}%")
        print(f"  RMSE              : {resultado['RMSE']:.2f}")
        print(f"  QWK               : {resultado['QWK']:.3f}")
        print(f"  Divergência (DIV) : {resultado['DIV']:.2f}%")
        print(f"  F1 Macro          : {resultado['F1-Macro']:.3f}")
        print(f"  F1 Weighted       : {resultado['F1-Weighted']:.3f}")
        print(f"  Agregado          : {resultado['Agregado']:.2f}")


In [ ]:
# === AVALIAÇÃO COM TODOS OS PROTOCOLOS ===
avaliacoes_por_modelo = {}
resumo_qwk = []
model_var_map = {
    "mbert": "df_base",
    "bertugues": "df_bertugues",
    "bertimbau": "df_bertimbau"
}

for model_key in ["mbert", "bertugues", "bertimbau"]:
    print(f"\n📊 Avaliando modelo: {model_key}")

    df_resultados = None
    var_name = model_var_map.get(model_key)
    
    if var_name and var_name in globals():
        try:
            df_resultados = globals()[var_name].copy()
        except NameError:
            pass

    if df_resultados is None:
        try:
            csv_path = os.path.join(SAVE_DIR, f"predicoes_{model_key}_finetuning_originals.csv")
            df_resultados = pd.read_csv(csv_path)
            print(f"  ℹ️ Usando dados do CSV")
        except (FileNotFoundError, pd.errors.EmptyDataError) as e:
            print(f"⚠️ Não foi possível encontrar dados para {model_key}. Pulando.")
            continue

    if "notas" in df_resultados.columns:
        df_resultados["notas"] = df_resultados["notas"].apply(
            lambda x: ast.literal_eval(x) if isinstance(x, str) else x
        )

    avaliacoes_por_modelo[model_key] = {}

    for protocol_key in PROTOCOL_ORDER:
        protocol_desc = PROTOCOL_LABELS[protocol_key]
        print(f"\n=== Protocolo: {protocol_desc} ===")
        avaliacoes = {}
        qwk_vals = []

        for c in competencias:
            comp_key = f"C{c}"
            
            if "notas" in df_resultados.columns:
                y_real = df_resultados["notas"].apply(lambda x: x[c-1] if isinstance(x, (list, tuple)) else np.nan)
            else:
                y_real = df_resultados[comp_key] if comp_key in df_resultados.columns else pd.Series(dtype=float)

            pred_col = f"pred_{comp_key}"
            if pred_col not in df_resultados.columns:
                print(f"⚠️ Coluna de previsão ausente para {comp_key}. Pulando.")
                continue
            y_pred = df_resultados[pred_col]

            pares = pd.DataFrame({"r": y_real, "p": y_pred}).dropna()
            if len(pares) == 0:
                continue

            y_r, y_p = apply_protocol(
                pares["r"].astype(int).tolist(),
                pares["p"].astype(int).tolist(),
                protocol_key
            )

            if not y_r:
                print(f"⚠️ Nenhum dado válido para {comp_key}")
                continue

            qwk_step = 20 if protocol_key == "no_change" else 40
            resultado = calcular_resultados(y_r, y_p, qwk_step=qwk_step)
            avaliacoes[comp_key] = resultado
            qwk_vals.append(resultado["QWK"])

            print(f"\n🔎 Avaliação - {comp_key}")
            print(f"  Acurácia       (ACC): {resultado['ACC']*100:.2f}%")
            print(f"  RMSE              : {resultado['RMSE']:.2f}")
            print(f"  QWK               : {resultado['QWK']:.3f}")
            print(f"  Divergência (DIV) : {resultado['DIV']:.2f}%")
            print(f"  F1 Macro          : {resultado['F1-Macro']:.3f}")
            print(f"  F1 Weighted       : {resultado['F1-Weighted']:.3f}")
            print(f"  Agregado          : {resultado['Agregado']:.2f}")

        avaliacoes_por_modelo[model_key][protocol_key] = avaliacoes

        if qwk_vals:
            resumo_qwk.append({
                "Modelo": model_key,
                "Esquema": protocol_desc,
                "QWK_médio": float(np.mean(qwk_vals)),
                "Qtd_comp_avaliadas": len(qwk_vals)
            })

if resumo_qwk:
    rank = pd.DataFrame(resumo_qwk).sort_values(by=["QWK_médio"], ascending=False)
    print("\n🏆 Ranking por QWK médio:")
    print(rank.to_string(index=False))
else:
    print("\n⚠️ Não foi possível compor o ranking.")


In [ ]:
# Calcular média dos QWK de cada modelo nos 5 protocolos
print("\n📊 Média dos QWK por Modelo (considerando todos os 5 protocolos):")
print("=" * 70)

medias_por_modelo = {}

for model_key in avaliacoes_por_modelo.keys():
    print(f"\n🔹 Modelo: {model_key.upper()}")
    print("-" * 70)

    # Calcular média por competência (nos 5 protocolos)
    medias_por_competencia = {}

    for c in competencias:
        comp_key = f"C{c}"
        qwk_comp = []

        # Coletar QWK desta competência em todos os protocolos
        for esquema_key in protocol_order:
            if esquema_key in avaliacoes_por_modelo[model_key]:
                avaliacoes = avaliacoes_por_modelo[model_key][esquema_key]
                if comp_key in avaliacoes and 'QWK' in avaliacoes[comp_key]:
                    qwk_comp.append(avaliacoes[comp_key]['QWK'])

        if qwk_comp:
            media_comp = np.mean(qwk_comp)
            medias_por_competencia[comp_key] = media_comp
            print(f"  {comp_key}: {media_comp:.4f} (média nos 5 protocolos)")

    # Calcular média geral (média das médias por competência)
    if medias_por_competencia:
        media_geral = np.mean(list(medias_por_competencia.values()))
        medias_por_modelo[model_key] = media_geral
        print(f"\n  📊 Média Geral: {media_geral:.4f} (média das médias por competência)")

print("\n" + "=" * 70)

# Criar DataFrame para visualização
if medias_por_modelo:
    df_medias = pd.DataFrame([
        {"Modelo": model, "Média_QWK": qwk}
        for model, qwk in medias_por_modelo.items()
    ]).sort_values(by="Média_QWK", ascending=False)

    print("\n📋 Tabela Resumo (Média Geral por Modelo):")
    print(df_medias.to_string(index=False))

In [ ]:
# === GERAR TABELAS LaTeX ===
metrics = [
    ("QWK", "QWK"),
    ("F1-Macro", "F1 Macro"),
    ("F1-Weighted", "F1 Weighted"),
]

for model_key in avaliacoes_por_modelo.keys():
    for met_key, met_title in metrics:
        latex_table = generate_latex_table(
            avaliacoes_por_modelo,
            model_key,
            met_key,
            met_title,
            competencias=competencias
        )
        print(f"\n=== Tabela LaTeX — {met_title} — {model_key} ===\n")
        print(latex_table)


In [ ]:
import matplotlib.pyplot as plt
import ast
from sklearn.metrics import r2_score, cohen_kappa_score
from sklearn.linear_model import LinearRegression
import numpy as np

# Lista de modelos para processar
modelos_info = [
    ("df_base", "mbert", "predicoes_mbert_finetuning_originals.csv"),
    ("df_bertugues", "bertugues", "predicoes_bertugues_finetuning_originals.csv"),
    ("df_bertimbau", "bertimbau", "predicoes_bertimbau_finetuning_originals.csv")
]

competencias = [1, 2, 3, 4, 5]

# Processar cada modelo disponível
modelos_processados = []

for df_var_name, modelo_nome, csv_filename in modelos_info:
    df_plot = None

    # Tentar usar variável local primeiro
    if df_var_name in globals():
        try:
            df_plot = globals()[df_var_name].copy()
            print(f"\n✅ Usando dados de {modelo_nome} da variável {df_var_name}")
        except:
            pass

    # Fallback: tentar ler do CSV
    if df_plot is None:
        try:
            csv_path = os.path.join(SAVE_DIR, csv_filename)
            df_plot = pd.read_csv(csv_path)
            print(f"✅ Carregando dados de {modelo_nome} do CSV: {csv_path}")
        except Exception as e:
            print(f"⚠️ Não foi possível carregar dados para {modelo_nome}: {e}")
            continue

    if df_plot is not None:
        # Processar coluna 'notas' se existir
        if "notas" in df_plot.columns:
            df_plot["notas"] = df_plot["notas"].apply(
                lambda x: ast.literal_eval(x) if isinstance(x, str) else x
            )

        r2_scores = {}
        qwk_scores = {}

        # Criar figura com 5 subplots lado a lado
        fig, axes = plt.subplots(1, 5, figsize=(30, 6))
        fig.suptitle(f'Notas Reais vs Preditas por Competência - Modelo {modelo_nome}',
                     fontsize=18, fontweight='bold', y=1.02)

        # Preparar dados para todos os gráficos primeiro
        dados_graficos = {}

        for c in competencias:
            comp_key = f"C{c}"

            # Extrair notas reais e preditas
            if "notas" in df_plot.columns:
                y_real = df_plot["notas"].apply(lambda x: x[c-1] if isinstance(x, (list, tuple)) else np.nan)
            else:
                y_real = df_plot[comp_key] if comp_key in df_plot.columns else pd.Series(dtype=float)

            y_pred = df_plot[f"pred_{comp_key}"]

            # Protocolo "sem ajuste" (no_change) - mesma lógica das tabelas
            pares = pd.DataFrame({"r": y_real, "p": y_pred}).dropna()
            y_real_clean = pares["r"].astype(int).values
            y_pred_clean = pares["p"].astype(int).values

            if len(y_real_clean) == 0:
                print(f"⚠️ Nenhum dado válido para {comp_key}")
                dados_graficos[comp_key] = None
                continue

            # Calcular QWK usando a mesma função das tabelas (protocolo "sem ajuste" com labels de 20 em 20)
            resultado = calcular_resultados(y_real_clean.tolist(), y_pred_clean.tolist(), qwk_step=20)
            qwk = resultado['QWK']
            qwk_scores[comp_key] = qwk

            # Calcular regressão linear: Y = nota prevista, X = nota real
            X = y_real_clean.reshape(-1, 1)  # X = nota real
            y = y_pred_clean  # Y = nota prevista
            reg = LinearRegression()
            reg.fit(X, y)
            slope = reg.coef_[0]
            intercept = reg.intercept_

            # Calcular R² usando score da regressão
            r2 = reg.score(X, y)
            r2_scores[comp_key] = r2

            # Gerar pontos para a linha de regressão (X = nota real, Y = nota prevista)
            x_line = np.array([0, 200])
            y_line = slope * x_line + intercept

            dados_graficos[comp_key] = {
                'y_real': y_real_clean,
                'y_pred': y_pred_clean,
                'r2': r2,
                'qwk': qwk,
                'slope': slope,
                'intercept': intercept,
                'x_line': x_line,  # X = nota real
                'y_line': y_line   # Y = nota prevista
            }

        # Criar os gráficos
        for idx, c in enumerate(competencias):
            comp_key = f"C{c}"
            ax = axes[idx]

            if dados_graficos[comp_key] is None:
                ax.text(0.5, 0.5, 'Sem dados', ha='center', va='center', transform=ax.transAxes)
                ax.set_title(f'{comp_key}', fontsize=14, fontweight='bold')
                continue

            dados = dados_graficos[comp_key]

            # Adicionar jitter aos pontos para melhor visualização (apenas para visualização)
            np.random.seed(42 + idx)  # Seed diferente para cada competência
            jitter_x = np.random.normal(0, 1.5, size=len(dados['y_pred']))
            jitter_y = np.random.normal(0, 1.5, size=len(dados['y_real']))

            y_real_jittered = dados['y_real'] + jitter_y  # X = nota real
            y_pred_jittered = dados['y_pred'] + jitter_x  # Y = nota prevista

            # Criar scatter plot com jitter (X = nota real, Y = nota prevista)
            ax.scatter(y_real_jittered, y_pred_jittered, alpha=0.5, s=30, edgecolors='black', linewidths=0.3)

            # Adicionar linha de referência (y = x)
            ax.plot([0, 200], [0, 200], 'r--', linewidth=2, label='y = x', alpha=0.8)

            # Adicionar linha de regressão linear (X = nota real, Y = nota prevista)
            ax.plot(dados['x_line'], dados['y_line'], 'b-', linewidth=2,
                    label=f'Regressão: y = {dados["slope"]:.3f}x + {dados["intercept"]:.2f}', alpha=0.8)

            # Configurar eixos (X = nota real, Y = nota prevista)
            ax.set_xlabel('Nota Real', fontsize=12, fontweight='bold')
            if idx == 0:
                ax.set_ylabel('Nota Prevista', fontsize=12, fontweight='bold')
            ax.set_title(f'{comp_key}\nR² = {dados["r2"]:.4f} | QWK = {dados["qwk"]:.4f}',
                         fontsize=13, fontweight='bold')

            # Configurar limites e ticks do eixo X (nota real: 0-200, de 20 em 20)
            ax.set_xlim(-10, 210)
            ax.set_xticks(range(0, 201, 20))
            ax.set_xticklabels(range(0, 201, 20), rotation=45, ha='right', fontsize=9)

            # Configurar limites e ticks do eixo Y (nota prevista: 0-200, de 40 em 40)
            ax.set_ylim(-10, 210)
            ax.set_yticks(range(0, 201, 40))
            ax.set_yticklabels(range(0, 201, 40), fontsize=9)

            # Grid
            ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.8)

            # Legenda
            ax.legend(loc='upper left', fontsize=8, framealpha=0.9)

            # Ajustar layout
            ax.set_aspect('equal', adjustable='box')

        plt.tight_layout()

        # Salvar gráfico completo
        plot_path = os.path.join(SAVE_DIR, f"graficos_real_vs_pred_{modelo_nome}_completo.png")
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"✅ Gráfico completo salvo em: {plot_path}")

        # Mostrar gráfico
        plt.show()

        # Imprimir resumo dos R² e QWK
        print(f"\n📊 Resumo das Métricas por Competência - {modelo_nome.upper()}:")
        print("=" * 60)
        print(f"{'Competência':<12} {'R²':<12} {'QWK':<12}")
        print("-" * 60)
        for comp_key in competencias:
            comp_key_str = f"C{comp_key}"
            if comp_key_str in r2_scores and comp_key_str in qwk_scores:
                print(f"{comp_key_str:<12} {r2_scores[comp_key_str]:<12.4f} {qwk_scores[comp_key_str]:<12.4f}")
        print("-" * 60)
        if r2_scores and qwk_scores:
            print(f"{'MÉDIO':<12} {np.mean(list(r2_scores.values())):<12.4f} {np.mean(list(qwk_scores.values())):<12.4f}")
        print("=" * 60)

        modelos_processados.append(modelo_nome)

# Resumo final
if modelos_processados:
    print(f"\n✅ Gráficos gerados para {len(modelos_processados)} modelo(s): {', '.join(modelos_processados)}")
else:
    print("\n❌ Nenhum modelo foi processado. Verifique se os dataframes ou CSVs estão disponíveis.")

In [ ]:
# === CONFUSION MATRICES ===
modelos_info = [
    ("df_base", "mbert", "predicoes_mbert_finetuning_originals.csv"),
    ("df_bertugues", "bertugues", "predicoes_bertugues_finetuning_originals.csv"),
    ("df_bertimbau", "bertimbau", "predicoes_bertimbau_finetuning_originals.csv")
]

for df_var_name, modelo_nome, csv_filename in modelos_info:
    print(f"\n{'='*80}")
    print(f"📊 CONFUSION MATRICES - Model: {modelo_nome.upper()}")
    print(f"{'='*80}")

    df_plot = None
    
    if df_var_name in globals():
        try:
            df_plot = globals()[df_var_name].copy()
            print(f"✅ Using data from {modelo_nome} from variable {df_var_name}")
        except:
            pass

    if df_plot is None:
        try:
            csv_path = os.path.join(SAVE_DIR, csv_filename)
            df_plot = pd.read_csv(csv_path)
            print(f"✅ Loading data from {modelo_nome} from CSV")
        except Exception as e:
            print(f"⚠️ Could not load data for {modelo_nome}: {e}")
            continue

    if df_plot is None:
        continue

    if "notas" in df_plot.columns:
        df_plot["notas"] = df_plot["notas"].apply(
            lambda x: ast.literal_eval(x) if isinstance(x, str) else x
        )

    metricas_por_competencia = {}

    for c in competencias:
        comp_key = f"C{c}"

        if "notas" in df_plot.columns:
            y_real = df_plot["notas"].apply(lambda x: x[c-1] if isinstance(x, (list, tuple)) else np.nan)
        else:
            y_real = df_plot[comp_key] if comp_key in df_plot.columns else pd.Series(dtype=float)

        y_pred = df_plot[f"pred_{comp_key}"]

        pares = pd.DataFrame({"r": y_real, "p": y_pred}).dropna()
        y_real_clean = pares["r"].astype(int).values
        y_pred_clean = pares["p"].astype(int).values

        if len(y_real_clean) == 0:
            continue

        plot_path = os.path.join(SAVE_DIR, f"confusion_matrix_{modelo_nome}_{comp_key}_finetuning_originals.png")
        metricas = plot_confusion_matrix(
            y_real_clean, y_pred_clean, comp_key,
            f"{modelo_nome} (Finetuning Originais)",
            save_path=plot_path,
            qwk_step=20
        )
        metricas_por_competencia[comp_key] = metricas
        print(f"✅ Confusion matrix for {comp_key} saved")

    print(f"\n📊 Summary of Metrics by Competence - {modelo_nome.upper()}:")
    print("=" * 120)
    print(f"{'Competence':<12} {'Accuracy':<10} {'QWK':<10} {'Precision-W':<12} {'Recall-W':<12} {'F1-Weighted':<12} {'F1-Macro':<12} {'Samples':<10}")
    print("-" * 120)
    for comp_key in competencias:
        comp_key_str = f"C{comp_key}"
        if comp_key_str in metricas_por_competencia:
            m = metricas_por_competencia[comp_key_str]
            print(f"{comp_key_str:<12} {m['accuracy']:<10.4f} {m['qwk']:<10.4f} {m['precision_weighted']:<12.4f} {m['recall_weighted']:<12.4f} "
                  f"{m['f1_weighted']:<12.4f} {m['f1_macro']:<12.4f} {m['total_samples']:<10}")
    print("-" * 120)
    if metricas_por_competencia:
        avg_acc = np.mean([m['accuracy'] for m in metricas_por_competencia.values()])
        avg_qwk = np.mean([m['qwk'] for m in metricas_por_competencia.values()])
        avg_f1_w = np.mean([m['f1_weighted'] for m in metricas_por_competencia.values()])
        avg_f1_m = np.mean([m['f1_macro'] for m in metricas_por_competencia.values()])
        total_samples = sum([m['total_samples'] for m in metricas_por_competencia.values()])
        print(f"{'AVERAGE':<12} {avg_acc:<10.4f} {avg_qwk:<10.4f} {'-':<12} {'-':<12} {avg_f1_w:<12.4f} {avg_f1_m:<12.4f} {total_samples:<10}")
    print("=" * 120)



